In [ ]:
from bs4 import BeautifulSoup as bs
import pandas as pd
pd.set_option('display.max_colwidth', 500)
import requests
import numpy as np
from math import *

access = True

# Crime Statistics from https://www.neighborhoodscout.com/
def data():
  city = str(input("What city do you plan to travel to? "))
  final_city = ""

  state = str(input("What state is the city in? "))
  stateWithSpace = state
  s = requests.get("https://worldpopulationreview.com/static/states/abbr-name-list.json")
  data2 = s.json()
  stateAbb = ""
  for x in range(len(data2)):
    if data2[x]["name"].lower() == state.lower():
      stateAbb = data2[x]["abbreviation"]  

  city = city.replace(" ", "-")
  page = requests.get("https://www.neighborhoodscout.com/" + stateAbb.lower() + "/" + city.lower() + "/crime")
  page
  # <Response [200]>
  soup = bs(page.content)
  soup
  crimeIndex = str(soup.find_all(class_='score'))
  crimeIndex = crimeIndex[19:]
  end = crimeIndex.index("<")
  crimeIndex = crimeIndex[:end]


  base_url = "https://api.openweathermap.org/geo/1.0/direct?q=" + city.lower() + ",USA&limit=10&appid=83d55d37ae3fbfcb341793fdd9a5a681"
  city = city.replace("-", " ")
  finalCity = ""
  newCity = []
  city = city.split(" ")
  for x in city:
    x = x.capitalize()
    newCity.append(x)
  for y in newCity:
    finalCity += y + " "
 
  testCrime = crimeIndex.strip()
  if testCrime == "":
    print(finalCity + "is too small to provide reliable safety information.")
  else:
    print(finalCity + "is safer than " + crimeIndex + "% of U.S. Cities.")
  #print(city.replace("-", " ").capitalize() + " is safer than " + crimeIndex + "% of U.S. Cities.")
  finalCity = finalCity.strip()
  '''
  # Population
  t = requests.get("https://api.census.gov/data/2019/pep/charagegroups?get=NAME,POP&for=state:*")
  data3 = t.json()
  for x in range(len(data3)):
    if data3[x][0].lower() == state.lower():
      population = data3[x][1]
    '''

  # Covid Statistics from https://www.nytimes.com/interactive/2021/us/covid-cases.html
  state = state.replace(" ", "-")
 
  r = requests.get(base_url)
  data4 = r.json()
  for x in range(len(data4)):
    if data4[x]['state'].lower().replace(" ", "-") == state.lower():
      og_lat = data4[x]['lat']
      og_lon = data4[x]['lon']
  s = requests.get("https://geo.fcc.gov/api/census/area?lat=" + str(og_lat) + "&lon=" + str(og_lon) + "&censusYear=2020&format=json")
  data1 = s.json()
  county = data1['results'][0]["county_name"]
  county = county.replace(" County", "")
  county = county.replace(" ", "-")
  if county == "New-York":
    county = "New-York-City"
  page = requests.get("https://www.nytimes.com/interactive/2021/us/" + county.lower() + "-" + state.lower() + "-covid-cases.html")
  page
  # <Response [200]>
  soup = bs(page.content)
  soup
  cases = str(soup.find_all(class_='svelte-ava34g'))
  first = cases.index("An average")
  cases = cases[first:]
  first = cases.index("> ")
  cases = cases[first + 2:]
  cases = cases[:cases.index(" ")]
  if county == "New-York-City":
    county = "New-York"
  print("Daily average of covid cases in " + county.replace("-", " ") + " County: " + cases)
  
  covid_cases = cases

  # Percentiles of Standard of Living
  q = requests.get("https://apps.bea.gov/api/data/?&UserID=F8D2A7DA-EF20-4F3D-925B-26167AE2788A&method=GetData&datasetname=Regional&TableName=SAINC1&GeoFIPS=STATE&LineCode=3&Year=2020&ResultFormat=JSON")
  data_cost = q.json()
  aList = []
  start = data_cost["BEAAPI"]["Results"]["Data"]
  for n in range(len(start)):
    temp = start[n]["GeoName"].strip("*")
    temp = temp.strip()
    if stateWithSpace.lower() == temp.lower():
      col = int(start[n]["DataValue"].replace(",",""))
    if n == 0 or n > 51:
      continue
    aList += [int(start[n]["DataValue"].replace(",",""))]
  aList.sort()
  median = np.percentile(aList,50)
  upper = np.percentile(aList,75)
  lower = np.percentile(aList,25)
  
  finalState = ""
  for n in range(len(stateWithSpace)):
    if n == 0:
      finalState += stateWithSpace[n].upper()
    else:
      if stateWithSpace[n-1] == " ":
        finalState += stateWithSpace[n].upper()
      else:
        finalState += stateWithSpace[n].lower()

  if col < lower:
    print("The annual cost of living in " + finalState + " is significantly low at $" + str(col) + ".")
  if lower <= col < median:
    print("The annual cost of living in " + finalState + " is somewhat low at $" + str(col) + ".")
  if col == median:
    print("The annual cost of living in " + finalState + " is median at $" + str(col) + ".")
  if median < col <= upper:
    print("The annual cost of living in " + finalState + " is somewhat high at $" + str(col) + ".")
  if upper < col:
    print("The annual cost of living in " + finalState + " is significantly high at $" + str(col) + ".")

  file = open("CountyPopulation.csv", encoding = "ISO-8859-1")
  header = file.readline()
  theList = file.readlines()
  file.close()

  s = requests.get("https://geo.fcc.gov/api/census/area?lat=" + str(og_lat) + "&lon=" + str(og_lon) + "&censusYear=2020&format=json")
  data1 = s.json()
  county = data1['results'][0]["county_name"]
  county = county.replace(" County", "")
  totalCases = 0
  caseList = []
  totalPop = 0
  popList = []
  countyList = []
  dLength = []
  dList = []
  stateList = []
  dtotal = 0

  for n in range(len(theList)):
    theList[n] = theList[n].split(",")
    if finalCity.lower() == "new york":
      countyPop = 1,628,706
      break
    elif theList[n][5].lower() == finalState.lower():
    
      if theList[n][5] == theList[n][6]:
        continue


      if theList[n][6].lower() == county.lower() + " county":
        countyPop = theList[n][18] 

  ####################
  file = open("uscities.csv")
  header = file.readline()
  cityList = file.readlines()
  file.close()
  r = 6373.0
  lat1 = radians(og_lat)
  lon1 = radians(og_lon)
  for n in range(len(cityList)):
    cityList[n] = cityList[n].split(",")
    for m in range(len(cityList[n])):
      cityList[n][m] = cityList[n][m].strip('"')
    if cityList[n][0].lower() == "brooklyn":
      continue
    if cityList[n][0].lower() == finalCity.lower() and cityList[n][3].lower() == finalState.lower():
      population = cityList[n][8]
    if cityList[n][3].lower().replace("-","") != finalState.lower().replace(" ","") or cityList[n][0].lower().replace("-","") == finalCity.lower().replace(" ",""):
      continue
    theCity = cityList[n][0]
    lat2 = radians(float(cityList[n][6]))
    lon2 = radians(float(cityList[n][7]))
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    d = r * c * 0.621371
    if d < 10:
      dtotal += d
      dLength += [d]
      dList += [cityList[n][0]]
      stateList += [cityList[n][3]]
      county1 = cityList[n][5]
      county1 = county1.replace(" ", "-")
      countyList += [county1]
      pop = int(cityList[n][8])
      popList += [pop]
      totalPop += pop
  if len(countyList) == 0:
    newCovid = "N/A"
    print("We do not predict an imminent increase in COVID cases from nearby areas.")
  else:
    for n in range(len(countyList)):
      page = requests.get("https://www.nytimes.com/interactive/2021/us/" + countyList[n].lower().replace(" ","-") + "-" + stateList[n].lower().replace(" ","-") + "-covid-cases.html")
      page
      # <Response [200]>
      soup = bs(page.content)
      soup
      cases = str(soup.find_all(class_='svelte-ava34g'))
      first = cases.index("An average")
      cases = cases[first:]
      first = cases.index("> ")
      cases = cases[first + 2:]
      cases = cases[:cases.index(" ")]
      totalCases += int(cases)
      caseList += [int(cases)]
    newCovid = totalCases/totalPop
    covid_rate = int(covid_cases) / int(population)
    alg1 = (newCovid - covid_rate)/(covid_rate)
    if alg1 > 3000:
      print("Potential risk for an increase in COVID due to high rates in nearby cities.")

    for n in range(len(countyList)):
      top1 = (10/(10 + float(dLength[n])))
      top2 = caseList[n] / popList[n] - covid_rate
      if top2 == 0:
        break
      alg2 = (top1/top2)/covid_rate
      if alg2 > 3000000:
        print("Potential risk for an increase in COVID due to high rates in {}.".format(countyList[n]))
        break

      
        
  
  


while access: 
  try:
    data()
    access = False
  except:
    print("Try Again")
    continue





Dc is safer than 4% of U.S. Cities.
Try Again
Chicago is safer than 9% of U.S. Cities.
Daily average of covid cases in Cook County: 934
The annual cost of living in Illinois is significantly high at $62930.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6fbef423-1ea7-40eb-a1a3-4dab894deb8f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>